# 19. Multi-Layer Perceptron (MLP) - Part 2
Now that I got MLP to work, I want to try it out on the complete dataset to see what happens to the results.

## Preprocessing

In [11]:
import pandas as pd
from preprocessing import PreProcessor

pp = PreProcessor()

df = pd.read_csv('Structured_DataFrame.csv', index_col=0)
df['Item Description'] = df['Item Description'].apply(lambda d: pp.preprocess(str(d)))
df

,Category,Item Description,category_id
0,Services/Hacking,12 Month HuluPlus gift Code 12-Month HuluPlus ...,0
1,Services/Hacking,Pay TV Sky UK Sky Germany HD TV and much mor...,0
2,Services/Hacking,OFFICIAL Account Creator Extreme 4.2 Tagged Su...,0
3,Services/Hacking,VPN > TOR > SOCK TUTORIAL How to setup a VPN >...,0
4,Services/Hacking,Facebook hacking guide . This guide will teac...,0
...,...,...,...
109585,Drugs/Opioids/Opium,1 gr purified Opium This Listing is for a gram...,95
109586,Weapons/Fireworks,Shipping Ticket in order for me to ship one of...,99
109587,Drugs/Opioids/Opium,0.50 GRAMS #4 White Afghani Heroin - FULL ESC...,95
109588,Drugs/Opioids/Opium,1.0 GRAMS #4 White Afghani Heroin - FULL ESCRO...,95


## Vectorizing

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2))
features = tfidf.fit_transform(df['Item Description'])
labels = df.Category

features

<109563x102238 sparse matrix of type '<class 'numpy.float64'>'
	with 3414110 stored elements in Compressed Sparse Row format>

## Splitting

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)

X_train

<73407x102238 sparse matrix of type '<class 'numpy.float64'>'
	with 2286840 stored elements in Compressed Sparse Row format>

## Creating the model
Compared to the part 1 MLP, I decreased the tolerance and max_iter so the model would stop earlier, because this one trains a lot slower due to the larger amount of data.

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=250, learning_rate='constant', 
                     alpha=0.001, solver='adam', verbose=1, random_state=21, tol=0.000001)
clf

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=250, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=21, shuffle=True, solver='adam', tol=1e-06,
       validation_fraction=0.1, verbose=1, warm_start=False)

## Training the model

In [5]:
clf.fit(X_train, y_train)

Iteration 1, loss = 1.81068031
Iteration 2, loss = 0.48601362
Iteration 3, loss = 0.28444859
Iteration 4, loss = 0.20646923
Iteration 5, loss = 0.16752440
Iteration 6, loss = 0.14723848
Iteration 7, loss = 0.13436415
Iteration 8, loss = 0.12439004
Iteration 9, loss = 0.11776405
Iteration 10, loss = 0.11191035
Iteration 11, loss = 0.11150976
Iteration 12, loss = 0.10616616
Iteration 13, loss = 0.10416760
Iteration 14, loss = 0.10389447
Iteration 15, loss = 0.10089518
Iteration 16, loss = 0.09882466
Iteration 17, loss = 0.09889388
Iteration 18, loss = 0.09772019
Iteration 19, loss = 0.09524186
Iteration 20, loss = 0.09523919
Iteration 21, loss = 0.09246504
Iteration 22, loss = 0.09121879
Iteration 23, loss = 0.09016739
Iteration 24, loss = 0.08947715
Iteration 25, loss = 0.08912988
Iteration 26, loss = 0.08657549
Iteration 27, loss = 0.08820899
Iteration 28, loss = 0.08731996
Iteration 29, loss = 0.08418626
Iteration 30, loss = 0.08392516
Iteration 31, loss = 0.08212144
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=250, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=21, shuffle=True, solver='adam', tol=1e-06,
       validation_fraction=0.1, verbose=1, warm_start=False)

## Validating the model

In [6]:
y_pred = clf.predict(X_test)

In [7]:
accuracy_score(y_test, y_pred)

0.8742670649408121

## Save the model

In [1]:
import pickle

filename = 'MLP_All_Model.sav'
pickle.dump(clf, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

NameError: name 'clf' is not defined

## Conclusion
The model performs quite well on the dataset. In notebook 21 I will compare some scores of the entire dataset and main categories to this one and other methods.